<a href="https://colab.research.google.com/github/Polar-Tsai/Lesson_Record/blob/main/0308_poem_naiveBayes_apply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 單純貝氏

當決策樹的欄位過多，即稱：「維度災難」
1. 演算法不能用
2. 資料量過少不足以用來分析欄位之間關係

文字問題，使用「貝氏定理」

字詞解釋: 
* P(A): A發生機率
* P(A∩B) or P(A,B): A發生且B發生機率
* 條件機率-- P(A|B) or P(A/B)
>* B「確定發生」(條件固定)，A發生機率
*　貝氏定理: P(A/B) = (P(A)*P(B/A))/P(B)

### 範例
> 欲測酒駕。測酒駕器具有95%準確，全台有酒駕者1%。求確定有酒駕機率?

> Ans. P(喝/器具響)=0.016
>* 結論: 其實很低...

## 單純貝氏
把非獨立假設為獨立，雖然這個假設是錯的，但因為在比大小情況下，大家錯的地方都一樣，依然有參考價值。
可以解決「維度災難」


In [ ]:
# 讀取詩詞csv檔
# 在github找到老師的檔案所在後，點選【raw】右鍵【複製連結網址】

from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0223/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0223/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")


('test.csv', <http.client.HTTPMessage at 0x7f01db69d690>)

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
# 不載入大辭典，因嘗試過不見得好，故只用jieba分析
# pd轉換技能
# type(train_df)
# type(train_df["作者"]) # 拿出一行= series
# 印的醜醜的是series
# 印的漂漂的是df

# 想要改欄名?
# Series.replace({"原本":"改成"}) 但太麻煩，所以用以下方法
# Series.unique/value_counts 這個多了解
u = train_df["作者"].unique()

# 方法一
# dic = {}
# for i in range(len(u)):
#   dic[d[i]] = i

# 方法二: enumerate
# 協助將作者的名字化成數字
name2cat = {name:i for i, name in enumerate(u)}
cat2name = {i:name for i, name in enumerate(u)} # 反向字典，將012轉成字典，類似尋找與取代的功能
y_train = train_df["作者"].replace(name2cat)
y_test = test_df["作者"].replace(name2cat)
y_test
y_train

0       0
1       0
2       0
3       0
4       0
       ..
2726    2
2727    2
2728    2
2729    2
2730    2
Name: 作者, Length: 2731, dtype: int64

In [ ]:
# a = int # int此類步驟可視為像「數字、字串、list...etc」，同理，也可將int設在變數a中
# a(5.2) # 透過a變數進行應用
# def test():
#   return int
# test()(5.3) # =int(5.3)，當最後面()加上去後，才正式執行；不加()，只是代表一個步驟

In [ ]:
# # apply
# # Series.apply(func)
# s = pd.Series([2,3,4])
# def test(s):
#   return s * s
# s.apply(test) # 不需要帶入，這裡的意思即s中的數字被代入了

In [ ]:
# apply正式應用
import jieba
def poemcut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["內容"].apply(poemcut) # 還不用執行，所以先不加()
x_test = test_df["內容"].apply(poemcut)
x_test

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.942 seconds.
Prefix dict has been built successfully.


0      日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1     朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2      李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3       故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4                 危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5                  床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6     天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
7               眾鳥 高 飛 儘 ， 孤雲獨 去 閒 。 相看 兩不厭 ， 隻 有 敬亭山 。
8     鳳凰 台上 鳳凰遊 ， 鳳去 台空江 自流 。 吳宮 花草 埋 幽徑 ， 晉代 衣冠 成古丘...
9     渡遠 荊門外 ， 來 從 楚國遊 。 山 隨 平野 儘 ， 江入 大荒 流 。 月 下 飛天...
10    百川 日東流 ， 客去 亦 不息 。 我 生苦 漂 蕩 ， 何時 有 終極 。 讚 公 釋 ...
11    細泉 兼 輕冰 ， 沮洳 棧道 濕 。 不辭 辛苦 行 ， 迫此 短景急 。 石門 雪雲隘 ...
12    首路 栗亭 西 ， 尚想 鳳凰村 。 季冬 攜 童稚 ， 辛苦 赴 蜀門 。 南登 木皮 嶺...
13    落日 在 簾 鉤 ， 溪邊 春事幽 。 芳菲 緣岸 圃 ， 樵 爨 倚灘 舟 。 啅 雀 爭...
14    竇侍 禦 ， 驥 之子 ， 鳳之雛 。 年 未 三十 忠義俱 ， 骨鯁 絕代 無 。 炯 如...
15    百草 競春華 ， 麗春應 最勝 。 少須 好 顏色 ， 多漫枝條 剩 。 紛紛 桃李 枝 ，...
16    何年 顧虎頭 ， 滿壁畫 瀛州 。 赤 日 石林 氣 ， 青天 江海 流 。 錫飛常 近鶴 ...
17    野寺 隱喬木 ， 山僧 高下 居 。 石門 日色異 ， 絳 氣橫 扶疏 。 窈窕 入

In [ ]:
# minmaxscaler是fit/transform形式 
# fit:學會min=0，max=100，目的:找出趨勢、找出有幾個欄位
# transform:把(所有點-min)/(max-min)，目的:進行表格內數數的任務
# 注意: 訓練資料fit+transform；測試資料只做transform

In [ ]:
# CountVectorizer協助數次數
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train) # 同時vec & fit
x_test_count = vec.transform(x_test)

In [ ]:
vec.vocabulary_ # 可以觀察欄位代表什麼
x_train_count
# sparse matrix稀疏矩陣: 適合用於龐大資料...，免於讓ram過勞
# vec.vocabulary_["，"] # keyerror: 代表已經被去除

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

Q. 有詞太常見，是不是該去除?

A. 不用，如果詞彙是大家都會使用，頻率一樣，那也不影響；
>* 如果頻率不同，就代表他有他的意義。
>* 所以常用詞不需要去除!!!


---


[naive bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
#### multinomial naive bayes多項式單純貝氏

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)
# alpha = (Nyi+a)/(Ny+a*n)
# Ny: 總用詞數
# Nyi: 特定詞詞數
# n: 欄位數
# a: 保底值，須注意不可給太大的值，通常0.01 or 0.001
# 為了避開機率為0or100(統計的大忌)，利用Smoothing，加保底值來避免
# 同時也須留意alpha不可太大，以免失真

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

pre = clf.predict(x_test_count)
print(accuracy_score(y_test, pre))
prob = clf.predict_proba(x_test_count)
print(log_loss(y_test, prob))


0.8333333333333334
1.1017983421769055


### confusion_matrix 混淆矩陣
#### 用以評估準確性

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index=[name + "(原本)" for name in u],
      columns=[name + "(預測)" for name in u])

,李白(預測),杜甫(預測),白居易(預測)
李白(原本),8,1,1
杜甫(原本),0,9,1
白居易(原本),1,1,8


In [ ]:
# 隨抓網路上詩詞比對
p = input("write:")
test = vec.transform([poemcut(p)])
prob = clf.predict_proba(test)[0] # 預測機率/[0]第一批
for i in range(len(u)):
  print(u[i], "的機率:", round(prob[i], 3))

write:雨落在屏東的甘蔗田裏，  甜甜的甘蔗甜甜的雨，  肥肥的甘蔗肥肥的田，  雨落在屏東肥肥的田裏。  從此地到山麓，  一大幅平原舉起  多少甘蔗，多少甘美的希冀！  長途車駛過青青的平原，  檢閱牧神青青的儀隊。  想牧神，多毛又多鬚，  在那一株甘蔗下午睡？     雨落在屏東的西瓜田裏，  甜甜的西瓜甜甜的雨，  肥肥的西瓜肥肥的田，  雨落在屏東肥肥的田裏。  從此地到海岸，  一大張河床孵出  多少西瓜，多少圓渾的希望！  長途車駛過纍纍的河床，  檢閱牧神纍纍的寶庫。  想牧神，多血又多子，  究竟坐在那一隻瓜上？     雨落在屏東的香蕉田裏，  甜甜的香蕉甜甜的雨，  肥肥的香蕉肥肥的田，     雨落在屏東肥肥的田裏。  雨是一首溼溼的牧歌，  路是一把瘦瘦的牧笛，  吹十里五里的阡阡陌陌。  雨落在屏東的香蕉田裏，  胖胖的香蕉肥肥的雨，  長途車駛不出牧神的轄區，  路是一把長長的牧笛。     正說屏東是最甜的縣，  屏東是方糖砌成的城，  忽然一個右轉，最鹹最鹹，  劈面撲過來那海。
李白 的機率: 1.0
杜甫 的機率: 0.0
白居易 的機率: 0.0
